In [5]:
import sqlalchemy as sa
import pandas as pd
import numpy as np
from notebooks.scripts import engine, game_types, team_names

In [6]:
meta_data = sa.MetaData(bind=engine)
sa.MetaData.reflect(meta_data)

In [7]:
Hero=meta_data.tables['Hero']
Map=meta_data.tables['Map']
ReplayDetail=meta_data.tables['ReplayDetail']
ReplayPlayer=meta_data.tables['ReplayPlayer']
TeamPlayerReplayHistory=meta_data.tables['TeamPlayerReplayHistory']

TeamRollup=meta_data.tables['TeamRollup']
TeamHeroRollup=meta_data.tables['TeamHeroRollup']
TeamStandingRollup=meta_data.tables['TeamStandingRollup']

Player=meta_data.tables['Player']

In [ ]:
player_history_df = pd.read_sql(sql=sa.select(TeamPlayerReplayHistory), con=engine)

replay_j = ReplayDetail\
    .join(Map,Map.c.MapId == ReplayDetail.c.MapId)\
    .join(TeamRollup,TeamRollup.c.PlayerId == ReplayPlayer.c.PlayerId)

replay_query = sa.select(
    TeamRollup.c.PlayerId
   , TeamRollup.c.Battletag
    , TeamRollup.c.AccountLevel
    , TeamRollup.c.TeamName
    , ReplayDetail.c.GameDate
    , ReplayDetail.c.GameType
    , ReplayDetail.c.GameLengthTimestamp
    , Hero.c.HeroName
    , Hero.c.NewRole
    , Map.c.MapName
    , ReplayDetail.c.ReplayFingerPrint
).select_from(replay_j).where(sa.and_(
    ReplayDetail.c.GameType.not_in(game_types)
))

replay_df = pd.read_sql(sql=replay_query, con=engine)

replay_df['RoleCode'] = replay_df['NewRole'].astype('category').cat.codes
replay_df['HeroCode'] = replay_df['HeroName'].astype('category').cat.codes
replay_df['MapCode'] = replay_df['MapName'].astype('category').cat.codes
replay_df['GameTypeCode'] = replay_df['GameType'].astype('category').cat.codes
replay_df['TeamNameCode'] = replay_df['TeamName'].astype('category').cat.codes
replay_df['FingerPrintCode'] = replay_df['ReplayFingerPrint'].astype('category').cat.codes


replay_df['Battletag'] = replay_df['Battletag'].str.lower()

print(f'Total full replay count: {replay_df.shape[0]}')

In [9]:
player_history_df = pd.read_sql(sql=sa.select(TeamPlayerReplayHistory), con=engine)

replay_j = ReplayDetail\
    .join(ReplayPlayer, ReplayPlayer.c.ReplayId == ReplayDetail.c.ReplayId)\
    .join(Hero,Hero.c.HeroId == ReplayPlayer.c.HeroId)\
    .join(Map,Map.c.MapId == ReplayDetail.c.MapId)\
    .join(TeamRollup,TeamRollup.c.PlayerId == ReplayPlayer.c.PlayerId)

replay_query = sa.select(
    TeamRollup.c.PlayerId
   , TeamRollup.c.Battletag
    , TeamRollup.c.AccountLevel
    , TeamRollup.c.TeamName
    , ReplayDetail.c.GameDate
    , ReplayDetail.c.GameType
    , ReplayDetail.c.GameLengthTimestamp
    , Hero.c.HeroName
    , Hero.c.NewRole
    , Map.c.MapName
    , ReplayPlayer.c.Team
    , ReplayPlayer.c.IsWinner
    , ReplayPlayer.c.Party
    , ReplayPlayer.c.FirstToTen
    , ReplayPlayer.c.Kills
    , ReplayPlayer.c.Assists
    , ReplayPlayer.c.Takedowns
    , ReplayPlayer.c.Deaths
    , ReplayPlayer.c.Level
    , ReplayPlayer.c.HeroDamage
    , ReplayPlayer.c.SiegeDamage
    , ReplayPlayer.c.StructureDamage
    , ReplayPlayer.c.MinionDamage
    , ReplayPlayer.c.CreepDamage
    , ReplayPlayer.c.SummonDamage
    , ReplayPlayer.c.TimeCcEnemyHeroes
    , ReplayPlayer.c.Healing
    , ReplayPlayer.c.SelfHealing
    , ReplayPlayer.c.DamageTaken
    , ReplayPlayer.c.ExperienceContribution
    , ReplayPlayer.c.HighestKillStreak
    , ReplayPlayer.c.TownKills
    , ReplayPlayer.c.Multikill
    , ReplayPlayer.c.TimeSpentDead
    , ReplayPlayer.c.MercCampCaptures
    , ReplayPlayer.c.WatchTowerCaptures
    , ReplayPlayer.c.MetaExperience
    , ReplayPlayer.c.ProtectionAllies
    , ReplayPlayer.c.SilencingEnemies
    , ReplayPlayer.c.RootingEnemies
    , ReplayPlayer.c.StunningEnemies
    , ReplayPlayer.c.ClutchHeals
    , ReplayPlayer.c.Escapes
    , ReplayPlayer.c.Vengeance
    , ReplayPlayer.c.OutnumberedDeaths
    , ReplayPlayer.c.TeamfightEscapes
    , ReplayPlayer.c.TeamfightHealing
    , ReplayPlayer.c.TeamfightDamageTaken
    , ReplayPlayer.c.TeamfightHeroDamage
    , ReplayPlayer.c.PhysicalDamage
    , ReplayPlayer.c.SpellDamage
    , ReplayPlayer.c.RegenGlobes
    , ReplayPlayer.c.TimeonFire
    , TeamRollup.c.BlizzardId
    , ReplayDetail.c.ReplayFingerPrint
).select_from(replay_j).where(sa.and_(
    ReplayDetail.c.GameType.not_in(game_types)
))

replay_df = pd.read_sql(sql=replay_query, con=engine)

replay_df['RoleCode'] = replay_df['NewRole'].astype('category').cat.codes
replay_df['HeroCode'] = replay_df['HeroName'].astype('category').cat.codes
replay_df['MapCode'] = replay_df['MapName'].astype('category').cat.codes
replay_df['GameTypeCode'] = replay_df['GameType'].astype('category').cat.codes
replay_df['TeamNameCode'] = replay_df['TeamName'].astype('category').cat.codes
replay_df['FingerPrintCode'] = replay_df['ReplayFingerPrint'].astype('category').cat.codes


replay_df['Battletag'] = replay_df['Battletag'].str.lower()

print(f'Total full replay count: {replay_df.shape[0]}')

Total full replay count: 51746 - Total player history count: 155760


In [ ]:
replay_df.to_sql('TeamReplayRollup', con=engine, if_exists='replace', index=False)

In [ ]:
team_replay_df = player_history_df[['ReplayFingerPrint', 'Battletag', 'Team']].groupby(['ReplayFingerPrint', 'Team']).agg('count').reset_index()
team_replay_df['TotalPlayers'] = team_replay_df['Battletag']
team_replay_df = team_replay_df[team_replay_df['Battletag'] >= 5]
t_df = pd.merge(player_history_df, team_replay_df[['ReplayFingerPrint', 'TotalPlayers']], left_on='ReplayFingerPrint', right_on='ReplayFingerPrint')
print(f'Total team replay count: {t_df.shape[0]}')
t_df.head(5000)

In [ ]:
t_df[t_df['TeamName'].isin(['30 seconds to mosh', 'gold dragons'])][['ReplayFingerPrint', 'TotalPlayers', 'Battletag', 'Team', 'GameDate', 'MapName', 'HeroName', 'GameType', 'IsWinner']].head(100)

In [ ]:
from datetime import datetime

print(datetime.now())